# EfficientDet Training On A Custom Dataset

### 0. Install Requirements

In [ ]:
!pip install pycocotools numpy==1.16.0 opencv-python tqdm tensorboard tensorboardX pyyaml webcolors matplotlib

### 1. Prepare Custom Dataset/Pretrained Weights (Skip this part if you already have datasets and weights of your own)

In [ ]:
# import os
# import sys
# if "projects" not in os.getcwd():
#   !git clone --depth 1 https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch
#   os.chdir('Yet-Another-EfficientDet-Pytorch')
#   sys.path.append('.')
# else:
#   !git pull

# # download and unzip dataset
# ! mkdir datasets
# ! wget https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch/releases/download/1.1/dataset_birdview_vehicles.zip
# ! unzip -d datasets/ dataset_birdview_vehicles.zip

# # download pretrained weights
# ! mkdir weights
# ! wget https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch/releases/download/1.0/efficientdet-d7.pth -O weights/efficientdet-d0.pth

# # prepare project file projects/logo.yml
# # showing its contents here
# ! cat projects/birdview_vehicles.yml

### 2. Training

In [1]:
# consider this is a simple dataset, train head will be enough.
%run train.py -c 0 -p cars --head_only False --batch_size 16 --load_weights weights/efficientdet-d0.pth  --num_epochs 12

# the loss will be high at first
# don't panic, be patient,
# just wait for a little bit longer

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[Warning] Ignoring Error(s) in loading state_dict for EfficientDetBackbone:
	size mismatch for classifier.header.pointwise_conv.conv.weight: copying a param with shape torch.Size([810, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([306, 64, 1, 1]).
	size mismatch for classifier.header.pointwise_conv.conv.bias: copying a param with shape torch.Size([810]) from checkpoint, the shape in current model is torch.Size([306]).
[Warning] Don't panic if you see this, this might be because you load a pretrained weights with different number of classes. The rest of the weights should be loaded already.
[Info] loaded weights: efficientdet-d0.pth, resuming checkpoint from step: 0


  0%|          | 0/364 [00:00<?, ?it/s]

Val. Epoch: 0/12. Classification loss: 3721.33228. Regression loss: 0.23592. Total loss: 3721.56820


  0%|          | 0/364 [00:00<?, ?it/s]

checkpoint...
Val. Epoch: 1/12. Classification loss: 689.17090. Regression loss: 0.19758. Total loss: 689.36848


...

checkpoint...
Val. Epoch: 9/12. Classification loss: 13.29599. Regression loss: 0.08744. Total loss: 13.38343


  0%|          | 0/364 [00:00<?, ?it/s]

checkpoint...
Val. Epoch: 10/12. Classification loss: 10.11176. Regression loss: 0.08213. Total loss: 10.19390


  0%|          | 0/364 [00:00<?, ?it/s]

Val. Epoch: 11/12. Classification loss: 7.69260. Regression loss: 0.07418. Total loss: 7.76678


### 3. Evaluation

In [2]:
#get latest weight file
# %cd logs/cars
# weight_file = !ls -Art | grep efficientdet
# %cd ../..

#uncomment the next line to specify a weight file
# weight_file[-1] = 'efficientdet-d0_11_4300.pth'

%run coco_eval.py -c 0 -p cars -w "logs/cars/efficientdet-d0_11_4368.pth"

running coco-style evaluation on project cars, weights logs/cars/efficientdet-d0_11_4368.pth...
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


  1%|▏         | 49/3400 [00:06<06:57,  8.04it/s]

In [13]:
import pandas as pd
import glob
import json

test_img_paths = sorted(glob.glob('./custom_dataset/test/*.png'))   
results = pd.read_csv('./custom_dataset/sample_submission.csv')   

with open("test_bbox_results.json","rb") as fr:
  json_data = json.load(fr)
  for i in json_data:
    filename = test_img_paths[int(i['image_id'])].split('/')[-1].split('\\')[-1] 
    label = int(i['category_id'])
    score = float(i['score'])
    bbox = i['bbox']
    x1, y1, x2, y2 = bbox[0] - bbox[2] / 2, bbox[1] - bbox[3] / 2, bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2
    results = results.append({
            "file_name" : filename,               
            "class_id" : label,                                                        
            "confidence" : score,                     
            "point1_x" : x1, "point1_y" : y1,
            "point2_x" : x2, "point2_y" : y1,
            "point3_x" : x2, "point3_y" : y2,
            "point4_x" : x1, "point4_y" : y2,
        }, ignore_index = True)
    
      
# for idx in range(len(test_img_paths)):                                                                            
#   prediction = data[idx]                 
#   #img = cv2.imread(test_img_paths[idx])
#   filename = test_img_paths[idx].split('/')[-1].split('\\')[-1]             
#   for i in range(34):
#     if prediction[i].shape[0]!=0:               
#       label = i                    
#       for j in range(prediction[i].shape[0]):
#         x1, y1, x2, y2 = prediction[i][j][0], prediction[i][j][1], prediction[i][j][2],prediction[i][j][3]
#         score = prediction[i][j][4]                
        
#         results = results.append({
#             "file_name" : filename,               
#             "class_id" : label,                                                        
#             "confidence" : score,                     
#             "point1_x" : x1, "point1_y" : y1,
#             "point2_x" : x2, "point2_y" : y1,
#             "point3_x" : x2, "point3_y" : y2,
#             "point4_x" : x1, "point4_y" : y2,
#         }, ignore_index = True)
    

In [15]:
results.to_csv('submit.csv', index = False)

### 4. Visualize

In [ ]:
import os
import cv2
pred = pd.read_csv('submit.csv')
img_path = './custom_dataset/test'
classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", 
               "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", 
               "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", 
               "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", 
               "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", 
               "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", 
               "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
               "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", 
               "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
               "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", 
               "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", 
               "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", 
               "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", 
               "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", 
               "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", 
               "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", 
               "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]

for i in range(len(pred)):
    scores = pred.iloc[i, :]
    label = "{}".format(classes[scores[1]]) + ' ' + str(scores[2])
    image_path = os.getcwd()+'/custom_dataset/test/'+scores[0]
    image = cv2.imread(image_path)
    cv2.rectangle(image, (int(scores[3]), int(scores[4])), (int(scores[7]), int(scores[8])), (0, 255, 0), 2)
    cv2.putText(image, label, (int(scores[3]), int(scores[4])-4), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        2 * 0.75, (0, 255, 0), 2, lineType=cv2.LINE_AA)
    cv2.imshow("Cars", image)
    cv2.waitKey()
    cv2.destroyAllWindows()